# Play MountainCar

In [ ]:
from gym.utils import play
import gym
env = gym.make("MountainCar-v0")
play.play(env)
env.close()

# Training

In [ ]:
from qlearning import *
import gym

def state_function(state):
    return (round(state[0], 1) , round(state[1], 2))
env = gym.make("MountainCar-v0")    
game = GamePlayer(env, state_function)

In [ ]:
total_episodes = 4000
alpha = 0.2                # Learning rate
gamma = 0.9                 # Discounting rate
decay_rate = 0.0005        # Exponential decay rate for exploration prob
epsilon = 0.9                 # Exploration rate
#game.erase_training()
rewards = game.train(total_episodes, alpha, gamma, epsilon, decay_rate, logEvery=1000)
print("Total reward average:", sum(rewards)/len(rewards))
print(len(game.qtable))

In [ ]:
action_function = lambda state: game.q_trained_action(state_function(state))
visualize_computer_playing(5, env, action_function)

# Using keras

In [ ]:
import importlib
import qlearning
importlib.reload(qlearning)
import gym

In [ ]:
env = gym.make("MountainCar-v0")
game = qlearning.GamePlayer(env)

In [ ]:
# Redefine rewards
import types

env.reset_backup = env.reset
env.step_backup = env.step
env.episode_step = 0

def reset(self):
    self.episode_step = 0
    return self.reset_backup()
    
def step(self, action):
    self.episode_step += 1
    state, reward, done, info = self.step_backup(action)
    if done and self.episode_step < 200:
        reward = 10+200-self.episode_step
    return state, reward, done, info

env.reset = types.MethodType(reset, env)
env.step = types.MethodType(step, env)

In [ ]:
N = 40
total_episodes = 500
game.keras_dqn_replay(N, total_episodes, alpha=0.1, gamma=0.9, layers_size=[24, 24, 24, 24],
                      decay_rate=0.9995, epsilon=0.8, logEvery=50)

In [ ]:
import numpy as np
batch = game.memory.sample(len(game.memory))
for state, action, reward, done, nstate in batch:
    if done and False:
        prediction = game.model.predict(np.array(state).reshape(1, 2))[0]
        print(state, action, nstate, reward, prediction)
    if reward >=0:
        prediction = game.model.predict(np.array(state).reshape(1, 2))[0]
        prediction_n = game.model.predict(np.array(nstate).reshape(1, 2))[0]
        print("Wow", state, action, reward, done, prediction, prediction_n)
len(game.memory)

In [ ]:
qlearning.visualize_computer_playing(5, env, game.keras_trained_action)